In [7]:
# Import neccessary libraries
import subprocess
import json
import os
from web3 import Web3
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from eth_account import Account
from constants import *
from dotenv import load_dotenv
load_dotenv()

# Import Mnemonic key from an environment variable
mnemonic = os.getenv('MNEMONIC', 'fame size ladder slide betray silver bless wreck raise nose spider metal')

# Create functions that derives the wallet keys 
def derive_wallets(mnemonic, coin, numderive):
    command = f'./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={numderive} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

# Store three constants in 'coins' variable 
coins = {
    ETH: derive_wallets(mnemonic, ETH, 3),
    BTCTEST: derive_wallets(mnemonic, BTCTEST, 3),
    BTC: derive_wallets(mnemonic, BTC, 3)
} 

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

private_key = os.getenv("PRIVATE_KEY")

# Initiate the private keys for ETH and BTCTEST 
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)

eth_priv_key = keys['eth'][0]['privkey']
btc_test_priv_key = keys['btc-test'][0]['privkey']

print(f" ETH Private Key: {eth_priv_key}")
print(f" BTC Test Private Key: {btc_test_priv_key}")

# Define function to convert the privkey string to an account object
def priv_key_to_account(coin, priv_key):
    private_key = os.getenv("PRIVATE_KEY")
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

# Create function that contains all metadata needed for transaction
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
        {"from": eth_account.address, "to": recipient, "value": amount}
    )
        return {
            "from": eth_account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_account.address)}
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

# Create function to sign the transaction
eth_account = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 6)[0]['privkey'])
btc_test_account = priv_key_to_account(BTCTEST,btc_test_priv_key)

def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, to, amount)
    
    if coin == ETH:
        signed_tx = eth_account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    elif coin == BTCTEST:
        tx_btc_test = create_tx(coin, account, to, amount)
        signed_tx = account.sign_transaction(tx)
        print(signed_tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

Python-dotenv could not parse statement starting at line 1


 ETH Private Key: 0xb1611113d2594b1aa820f204521ef3283c7044811438c2b4d978554316a9122c
 BTC Test Private Key: cRMm5qqp8F1iSjshDZygHoNpg4MQxUHhLhsRdXhensQWLvdAvpfY


## BTC-TEST Transactions

In [8]:
create_tx(BTCTEST,btc_test_account,"mxCiNVhZza6oJ7ynoahArJDAqEarPEmu3y", 0.00001)

'{"unspents":[{"amount":1770052,"confirmations":0,"script":"76a914b70707405e596e7948efc0d6a5900521986e6c6288ac","txid":"c5dbbb30ab3f774758c42a399c6e58817362a0c0f3e2f7719e478767e3e6b4b6","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mxCiNVhZza6oJ7ynoahArJDAqEarPEmu3y",1000],["mxCiNVhZza6oJ7ynoahArJDAqEarPEmu3y",1746000]]}'

In [9]:
send_tx(BTCTEST,btc_test_account,"mmYd98EUReS6kJUF2vtyAYWe2RrBbyuG2C", 0.00001)

0100000001b6b4e6e36787479e71f7e2f3c0a0627381586e9c392ac45847773fab30bbdbc5010000006a4730440220294a1837adcb23f7e2e4ce415e099ccc6bc22f89923981768b5c77a0d6f3bedc02203507e92b3e9a3a76ddacebdc9101fd618ff9a8892b796734f29588261e0575f0012103a0d9bd970ecdce0977d43c76e6b7b83b12d65cfb8fce72a899648f8ed558e57affffffff02e8030000000000001976a9144221990457164d47298efd46fa5d2b95c85e292288ac50a41a00000000001976a914b70707405e596e7948efc0d6a5900521986e6c6288ac00000000
